🚌 Projet MDM - Mobilité Durable en Montagne ⛰️

*Author: Laurent Sorba*

*Date: 14/09/2025*

**Description :**

This notebook combines up to 3 stop files (GeoParquets or GeoJSONs), reproject inputs to EPSG:3857 if needed,
cluster by proximity, represent cluster by centroid or mediod, and plot the result on an interactive map.

See https://github.com/data-for-good-grenoble/mobilite_durable/issues/37

In [ ]:
import folium
import geopandas as gpd


def plot_clusters_interactive(
    agg_gdf: gpd.GeoDataFrame,
    title: str = None,
    point_radius: int = 5,
    tiles: str = "OpenStreetMap",
):
    """
    Display an interactive Leaflet map (zoom/scroll) using folium.
    - agg_gdf: GeoDataFrame with columns [geometry(Point in EPSG:3857), colour, name(optional)]
    - title: optional map title (added as child HTML)
    - point_radius: circle marker radius in pixels
    - tiles: folium base tiles name or URL template
    Returns the folium.Map object, which renders in notebooks.
    """

    # Convert to WGS84 for web maps
    gdf_ll = agg_gdf.to_crs(EPSG_WGS84)

    # Determine initial center
    if not gdf_ll.empty:
        center = [gdf_ll.geometry.y.mean(), gdf_ll.geometry.x.mean()]
    else:
        center = [45.1885, 5.7245]  # Grenoble as default

    m = folium.Map(location=center, zoom_start=8, tiles=tiles)

    # Optionally add a title
    if title:
        title_html = f'<h4 style="position: fixed; top: 10px; left: 50px; z-index: 9999; background: rgba(255,255,255,0.8); padding: 6px 10px; margin: 0;">{title}</h4>'
        m.get_root().html.add_child(folium.Element(title_html))

    # Group by colours for consistency
    colour_labels = [
        ("green", "in A+B+C (green)"),
        ("blue", "in any 2 sources (blue)"),
        ("orange", "only in TDG (orange)"),
        ("yellow", "only in OSM (yellow)"),
        ("brown", "only in C2C (brown)"),
        ("gray", "none/other (gray)"),
    ]

    bounds = []
    for colour, label in colour_labels:
        sub = gdf_ll[gdf_ll["colour"] == colour]
        if len(sub) == 0:
            continue
        fg = folium.FeatureGroup(name=label, show=True)
        for _, row in sub.iterrows():
            pt = row.geometry
            popup_txt = []

            if "names" in row:
                popup_txt.append(f"names: {','.join(row['names'])}")
            if "ids" in row:
                popup_txt.append(f"ids: {','.join(row['ids'])}")
            if "cluster" in row:
                popup_txt.append(f"cluster: {row['cluster']}")
            if "agencies" in row:
                popup_txt.append(f"agencies: {','.join(row['agencies'])}")
            if "sources" in row:
                popup_txt.append(f"sources: {','.join(row['sources'])}")
            if "count" in row:
                popup_txt.append(f"count: {row['count']}")

            popup = "<br>".join(popup_txt) if popup_txt else None

            folium.CircleMarker(
                location=(pt.y, pt.x),
                radius=point_radius,
                color=colour,
                fill=True,
                fill_opacity=0.8,
                fill_color=colour,
                popup=popup,
            ).add_to(fg)
            bounds.append((pt.y, pt.x))
        fg.add_to(m)

    folium.LayerControl(collapsed=False).add_to(m)

    # Fit bounds if we have any points
    if bounds:
        m.fit_bounds(bounds, padding=(20, 20))

    return m

In [ ]:
import unicodedata
from pathlib import Path
from typing import Sequence

import numpy as np
import pandas as pd
from shapely.geometry import MultiPoint
from shapely.geometry.point import Point
from sklearn.cluster import DBSCAN

from src.settings import EPSG_WEB_MERCATOR, EPSG_WGS84

# Default colour mapping
"""
Colours:
- green = in A+B+C
- blue  = in any 2 sources
- orange/yellow/brown = only in TDG / OSM / C2C
"""

COLOUR_SINGLE = {"TDG": "orange", "OSM": "yellow", "C2C": "brown"}
COLOUR_TWO = "blue"
COLOUR_THREE = "green"
COLOUR_NONE = "gray"

TARGET_EPSG = EPSG_WEB_MERCATOR  # metric CRS for clustering and output


def read_and_ensure_crs(path: str, label: str, target_epsg: str = TARGET_EPSG):
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"{path} not found")

    # Choose reader based on extension (GeoJSON vs GeoParquet)
    suffix = p.suffix.lower()
    if suffix in {".parquet", ".pq", ".geoparquet", ".gpq"}:
        gdf = gpd.read_parquet(str(p))
    else:
        gdf = gpd.read_file(str(p))

    gdf = gdf[gdf.geometry.type == "Point"].copy()
    if gdf.empty:
        raise ValueError(f"{path} contains no Point geometries")
    if not gdf.crs:
        # OSM data has no CRS defined, so we assume it's WGS84
        gdf.set_crs(EPSG_WGS84, inplace=True)

    # reproject to target if needed
    if gdf.crs != target_epsg:
        gdf = gdf.to_crs(target_epsg)

    gdf["source"] = label

    def _strip_accents(s: str) -> str:
        # Robust to NaN/None and vectorised via .map below
        if pd.isna(s):
            return ""
        nfkd = unicodedata.normalize("NFKD", str(s))
        return "".join(c for c in nfkd if not unicodedata.combining(c))

    # Normalise stop name column: unify 'name' and 'stop_name' into 'stop_name'
    try:
        if "stop_name" in gdf.columns and "name" in gdf.columns:
            # Prefer non-empty stop_name, otherwise fall back to name
            sn = gdf["stop_name"].astype(str)
            nm = gdf["name"].astype(str)
            use_sn = sn.str.strip().astype(bool)
            stop_name_series = sn.where(use_sn, nm)
        elif "stop_name" not in gdf.columns and "name" in gdf.columns:
            stop_name_series = gdf["name"].astype(str)
        else:
            stop_name_series = gdf.get("stop_name", pd.Series("", index=gdf.index)).astype(str)

        # Vectorised normalisation
        stop_name_norm = stop_name_series.map(_strip_accents).str.lower().str.strip()

        # Case-insensitive canonicalization
        key_ci = stop_name_norm.str.casefold()
        canon_map = stop_name_norm.groupby(key_ci).first()
        gdf["stop_name"] = key_ci.map(canon_map)

    except Exception:
        pass

    # Normalise agency_name and network
    try:
        if "agency_name" in gdf.columns and "network" in gdf.columns:
            gdf["agency_name"] = gdf["network"]
        elif "agency_name" not in gdf.columns and "network" in gdf.columns:
            gdf["agency_name"] = gdf["network"]
        # else: keep existing agency_name as is
    except Exception:
        pass

    # Normalise stop id column: unify 'id' and 'stop_id' into 'stop_id'
    try:
        if "stop_id" in gdf.columns and "id" in gdf.columns:
            gdf["stop_id"] = gdf["id"].fillna("").astype(str)
        elif "stop_id" not in gdf.columns and "gtfs_id" in gdf.columns:
            gdf["stop_id"] = gdf["gtfs_id"].fillna("").astype(str)
        elif "stop_id" not in gdf.columns and "osm_id" in gdf.columns:
            gdf["stop_id"] = gdf["osm_id"].fillna("").astype(str)
        # else: keep existing stop_id as is
    except Exception:
        pass
    return gdf


def medoid_point(points: Sequence[Point]) -> Point:
    coords = np.array([[p.x, p.y] for p in points])
    if len(coords) == 1:
        return points[0]
    D = np.sqrt(((coords[:, None, :] - coords[None, :, :]) ** 2).sum(axis=2))
    idx = D.sum(axis=1).argmin()
    return points[int(idx)]


def cluster_and_represent(gdf_all: gpd.GeoDataFrame, tolerance: float, use_medoid: bool):
    # gdf_all is expected in TARGET_EPSG (metric)
    coords = np.array([[pt.x, pt.y] for pt in gdf_all.geometry])
    db = DBSCAN(eps=tolerance, min_samples=1, metric="euclidean").fit(coords)
    gdf_all = gdf_all.copy()
    gdf_all["cluster"] = db.labels_
    clusters = []
    for cid, sub in gdf_all.groupby("cluster"):
        geoms = list(sub.geometry)
        sources = sorted(sub["source"].unique().tolist())
        ids = sub["stop_id"].tolist() if "stop_id" in sub.columns else list(sub.index)
        agencies = sorted(
            {str(x).strip() for x in sub["agency_name"].dropna() if str(x).strip()}
        )
        names = sorted({str(x).strip() for x in sub["stop_name"].dropna() if str(x).strip()})
        rep_pt = medoid_point(geoms) if use_medoid else MultiPoint(geoms).centroid

        clusters.append(
            {
                "cluster": int(cid),
                "geometry": rep_pt,
                "ids": ids,
                "agencies": agencies,
                "count": len(sub),
                "sources": sources,
                "names": names,
            }
        )
    agg = gpd.GeoDataFrame(clusters, geometry="geometry", crs=gdf_all.crs)

    def pick_colour(sources: Sequence[str]) -> str:
        n = len(sources)
        if n >= 3:
            return COLOUR_THREE
        if n == 2:
            return COLOUR_TWO
        if n == 1:
            return COLOUR_SINGLE.get(sources[0], COLOUR_NONE)
        return COLOUR_NONE

    agg["colour"] = agg["sources"].apply(pick_colour)
    agg["in_TDG"] = agg["sources"].apply(lambda s: "TDG" in s)
    agg["in_OSM"] = agg["sources"].apply(lambda s: "OSM" in s)
    agg["in_C2C"] = agg["sources"].apply(lambda s: "C2C" in s)
    return agg

## Explanation of Medoid and Centroid

The *medoid* is the point that minimises the total distance to all other points in a dataset, making it robust to outliers. In contrast, the *centroid* is the average of the coordinates of all points, but it can be influenced by extreme values and does not always correspond to an actual point in the dataset.

In [ ]:
# Set the input GeoJSONs or GeoParquet (can be 1 to 3 sources)
a_path = Path("../data/transportdatagouv/stops_38.parquet")
b_path = Path("../data/OSM/bus_stops_isere.parquet")
c_path = None  # Path("../data/C2C/
tol_m = 200.0  # in meters
use_medoid = True

gdfs = []
for pth, lab in ((a_path, "TDG"), (b_path, "OSM"), (c_path, "C2C")):
    if pth:
        gdfs.append(read_and_ensure_crs(pth, lab, target_epsg=TARGET_EPSG))
gdf_all = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True, sort=False), crs=gdfs[0].crs)
res = cluster_and_represent(gdf_all, tol_m, use_medoid)
out_nb = res[
    [
        "geometry",
        "cluster",
        "ids",
        "names",
        "agencies",
        "count",
        "sources",
        "in_TDG",
        "in_OSM",
        "in_C2C",
        "colour",
    ]
]

In [ ]:
m = plot_clusters_interactive(out_nb, title="Stops diff clusters (coloured by presence)")
m  # display in notebook